# OTMapper test

In [295]:
import OTMapOnto as maponto
import OTMapper as otmapper
import OTNeighborhood_TDA as mapneighbor
from rdflib import Graph

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%%time
model_path="../model/crawl-300d-2M-subword.bin"
embs_model = maponto.load_embeddings(model_path, None)

INFO:root:Load pre-trained embeddings for about 10 mins if the model hasn't been load yet.
INFO:root:Loading the pre-trained Fasttext model...Please be patient...It may take about 10 mins to load...
INFO:gensim.models._fasttext_bin:loading 2000000 words for fastText model from ../model/crawl-300d-2M-subword.bin


Loading the pre-trained Fasttext model...Please be patient...It may take about 10 mins to load...


ERROR:gensim.models._fasttext_bin:failed to decode invalid unicode bytes b'DeutschHrvatskiEnglishDanskNederlandssuomiFran\xc3\xa7ais\xce\x95\xce\xbb\xce\xbb\xce'; replacing invalid characters, using 'DeutschHrvatskiEnglishDanskNederlandssuomiFrançaisΕλλ\\xce'
ERROR:gensim.models._fasttext_bin:failed to decode invalid unicode bytes b'\xe3\x81\x99\xe3\x81\xb9\xe3\x81\xa6\xe3\x81\xae\xe5\x9b\x9e\xe7\xad\x94\xe3\x82\x92\xe9\x9d\x9e\xe8\xa1\xa8\xe7\xa4\xba\xe3\x81\xab\xe3\x81\x99\xe3\x82\x8b\xe8\xb3\xaa\xe5\x95\x8f\xe3\x82\x92\xe5\x89\x8a\xe9\x99\xa4\xe3\x81\x97\xe3'; replacing invalid characters, using 'すべての回答を非表示にする質問を削除し\\xe3'
ERROR:gensim.models._fasttext_bin:failed to decode invalid unicode bytes b'00Z\xe9\x83\xa8\xe5\xb1\x8b\xe3\x82\xbf\xe3\x82\xa4\xe3\x83\x97\xe3\x81\xbe\xe3\x82\x8b\xe3\x81\xbe\xe3\x82\x8b\xe8\xb2\xb8\xe5\x88\x87\xe5\xbb\xba\xe7\x89\xa9\xe3\x82\xbf\xe3\x82\xa4\xe3\x83\x97\xe4\xb8\x80\xe8\xbb\x92\xe5'; replacing invalid characters, using '00Z部屋タイプまるまる貸切建物タイプ一軒\\xe5'
E

Completed loading the pre-trained Fasttext model.
CPU times: user 3min 55s, sys: 11.7 s, total: 4min 7s
Wall time: 4min 7s


In [4]:
mapper = otmapper.Mapper(embs_model)

# MSE Benchmark

In [322]:
import importlib
importlib.reload(maponto)
importlib.reload(mapneighbor)
importlib.reload(otmapper)

INFO:root:logging info test


<module 'OTMapper' from '/home/ubuntu/django/otmaponto/match/ontology_mapping/src/OTMapper.py'>

In [66]:
source_url = "../data/MSE-Benchmark/testCases/firstTestCase/MaterialInformation-Reduced.owl"
target_url = "../data/MSE-Benchmark/testCases/firstTestCase/MatOnto.owl"
output_url = '../data/results/MSE_firstTest_case_alignments.rdf'
refs_url = "../data/MSE-Benchmark/testCases/firstTestCase/RefAlign1.rdf"

In [67]:
mapper.align(source_url, target_url, output_url)

INFO:root:Python ensemble mapper info: map ../data/MSE-Benchmark/testCases/firstTestCase/MaterialInformation-Reduced.owl to ../data/MSE-Benchmark/testCases/firstTestCase/MatOnto.owl
INFO:root:Read source with 309 triples.


../data/MSE-Benchmark/testCases/firstTestCase/MaterialInformation-Reduced.owl
../data/MSE-Benchmark/testCases/firstTestCase/MatOnto.owl


INFO:root:Read target with 11363 triples.
INFO:root:MAP CONCEPTS
INFO:root:The number of level_1 predicted mapping is 2.
INFO:root:Retrieving Synsets by WordNet...
INFO:root:The number of level_2 predicted mapping is 5
INFO:root:Compute the related words for each source concept....
25it [00:00, 54.04it/s]
INFO:root:Total number of source concepts computed with relatd words is 25
INFO:root:Compute the related words for each target concept....
820it [00:17, 47.73it/s]
INFO:root:Total number of target concepts computed with related words is 818
100%|██████████| 25/25 [00:00<00:00, 268.50it/s]
INFO:root:The number of level_3 predicted mapping is 10
INFO:root:Compute Pairwise Wasserstein Distances...
100%|██████████| 15/15 [00:15<00:00,  1.02s/it]
INFO:root:The number of level_4 predicted mapping is 2
INFO:root:Skip level_5 mapping.
INFO:root:Matching Label Embeddings by Optimal Transport...
INFO:root:Computing the Ground Embedding Costs between the Source and Target Points...
INFO:root:The

INFO:root:Compute the related words for each source concept....
9it [00:00, 57.82it/s]
INFO:root:Total number of source concepts computed with relatd words is 9
INFO:root:Compute the related words for each target concept....
80it [00:01, 52.08it/s]
INFO:root:Total number of target concepts computed with related words is 80
100%|██████████| 9/9 [00:00<00:00, 2569.86it/s]
INFO:root:The number of level_3 predicted mapping is 2
INFO:root:Skip the level_4 mapping.
INFO:root:Skip the level_5 mapping.
INFO:root:Skip the level_6 mappping.
INFO:root:TOTAL NUMBER OF MAPPINGS BETWEEN OBJECT PROPERTIES IS 4
INFO:root:========================================================
INFO:root:MAP DATATYPE PROPERTIES
INFO:root:Retrieving Synsets by WordNet...
INFO:root:Compute the related words for each source concept....
32it [00:00, 55.12it/s]
INFO:root:Total number of source concepts computed with relatd words is 32
INFO:root:Compute the related words for each target concept....
13it [00:00, 44.87it/s]
IN

In [68]:
preds = maponto.load_alignments(output_url, None)
maponto.evaluate(preds, refs_url)

Total number of None is 39
Total number of references is 23
Total correctly predicted alignments is 9
Total number of predicted is 39
Precision is 0.23076923076923078
Recall is 0.391304347826087
F1-Measure is 0.2903225806451613


In [69]:
source_url = "../data/MSE-Benchmark/testCases/secondTestCase/MaterialInformation.owl"
target_url = "../data/MSE-Benchmark/testCases/secondTestCase/MatOnto.owl"
output_url = '../data/results/MSE_secondTest_case_alignments.rdf'
refs_url = "../data/MSE-Benchmark/testCases/secondTestCase/RefAlign2.rdf"

In [70]:
mapper.align(source_url, target_url, output_url)

INFO:root:Python ensemble mapper info: map ../data/MSE-Benchmark/testCases/secondTestCase/MaterialInformation.owl to ../data/MSE-Benchmark/testCases/secondTestCase/MatOnto.owl


../data/MSE-Benchmark/testCases/secondTestCase/MaterialInformation.owl


INFO:root:Read source with 2350 triples.


../data/MSE-Benchmark/testCases/secondTestCase/MatOnto.owl


INFO:root:Read target with 11362 triples.
INFO:root:MAP CONCEPTS
INFO:root:The number of level_1 predicted mapping is 59.
INFO:root:Retrieving Synsets by WordNet...
INFO:root:The number of level_2 predicted mapping is 209
INFO:root:Compute the related words for each source concept....
362it [00:06, 55.32it/s]
INFO:root:Total number of source concepts computed with relatd words is 362
INFO:root:Compute the related words for each target concept....
643it [00:12, 52.36it/s]
INFO:root:Total number of target concepts computed with related words is 641
100%|██████████| 362/362 [00:01<00:00, 324.94it/s]
INFO:root:The number of level_3 predicted mapping is 67
INFO:root:Compute Pairwise Wasserstein Distances...
100%|██████████| 295/295 [03:38<00:00,  1.35it/s]
INFO:root:The number of level_4 predicted mapping is 10
INFO:root:Skip level_5 mapping.
INFO:root:Matching Label Embeddings by Optimal Transport...
INFO:root:Computing the Ground Embedding Costs between the Source and Target Points...
INF

In [46]:
preds = maponto.load_alignments(output_url, None)
maponto.evaluate(preds, refs_url)

Total number of None is 413
Total number of references is 63
Total correctly predicted alignments is 57
Total number of predicted is 413
Precision is 0.13801452784503632
Recall is 0.9047619047619048
F1-Measure is 0.23949579831932774


In [43]:
source_url = "../data/MSE-Benchmark/testCases/thirdTestCase/MaterialInformation.owl"
target_url = "../data/MSE-Benchmark/testCases/thirdTestCase/EMMO.owl"
output_url = '../data/results/MSE_thirdTest_case_alignments.rdf'
refs_url = "../data/MSE-Benchmark/testCases/thirdTestCase/RefAlign3.rdf"

In [44]:
mapper.align(source_url, target_url, output_url)

INFO:root:Python ensemble mapper info: map ../data/MSE-Benchmark/testCases/thirdTestCase/MaterialInformation.owl to ../data/MSE-Benchmark/testCases/thirdTestCase/EMMO.owl


../data/MSE-Benchmark/testCases/thirdTestCase/MaterialInformation.owl


INFO:root:Read source with 2350 triples.


../data/MSE-Benchmark/testCases/thirdTestCase/EMMO.owl
http://emmo.info/emmo/1.0.0-alpha2/middle/isq
http://emmo.info/emmo/1.0.0-alpha2/middle/metrology
http://emmo.info/emmo/1.0.0-alpha2/middle/math
http://emmo.info/emmo/1.0.0-alpha2/middle/reductionistic
http://emmo.info/emmo/1.0.0-alpha2/middle/materials
http://emmo.info/emmo/1.0.0-alpha2/middle/models
http://emmo.info/emmo/1.0.0-alpha2/top
http://emmo.info/emmo/1.0.0-alpha2/top/physical
http://emmo.info/emmo/1.0.0-alpha2/top/mereotopology
http://emmo.info/emmo/1.0.0-alpha2/top/annotations
http://emmo.info/emmo/1.0.0-alpha2/middle/physicalistic
http://emmo.info/emmo/1.0.0-alpha2/middle/siunits
http://emmo.info/emmo/1.0.0-alpha2/middle/properties
http://emmo.info/emmo/1.0.0-alpha2/middle/perceptual
http://emmo.info/emmo/1.0.0-alpha2/middle/holistic
http://emmo.info/emmo/1.0.0-alpha2/middle/manufacturing
http://emmo.info/emmo/1.0.0-alpha2/middle/semiotics
http://emmo.info/emmo/1.0.0-alpha2/middle/units-extension


INFO:root:Read target with 4644 triples.
INFO:root:MAP CONCEPTS
INFO:root:The number of level_1 predicted mapping is 52.
INFO:root:Retrieving Synsets by WordNet...
INFO:root:The number of level_2 predicted mapping is 87
INFO:root:Compute the related words for each source concept....
445it [00:08, 50.18it/s]
INFO:root:Total number of source concepts computed with relatd words is 445
INFO:root:Compute the related words for each target concept....
158it [00:03, 52.05it/s]WARNING:rdflib.term:http://emmo.info/emmo#EMMO_28fbea28_2204_4613_87ff_6d877b855fcd  does not look like a valid URI, trying to serialize this will break.
357it [00:07, 50.71it/s]
INFO:root:Total number of target concepts computed with related words is 357
100%|██████████| 445/445 [00:00<00:00, 550.82it/s]
INFO:root:The number of level_3 predicted mapping is 55
INFO:root:Compute Pairwise Wasserstein Distances...
100%|██████████| 390/390 [02:00<00:00,  3.24it/s]
INFO:root:The number of level_4 predicted mapping is 7
INFO:ro

INFO:root:The number of level_6 predicted mapping is 209
INFO:root:TOTAL NUMBER OF MAPPINGS BETWEEN CONCPETS IS 410
INFO:root:=================================================
INFO:root:MAP OBJECT PROPERTIES
INFO:root:Retrieving Synsets by WordNet...
INFO:root:Compute the related words for each source concept....
28it [00:00, 56.14it/s]
INFO:root:Total number of source concepts computed with relatd words is 28
INFO:root:Compute the related words for each target concept....
31it [00:00, 48.12it/s]
INFO:root:Total number of target concepts computed with related words is 31
100%|██████████| 28/28 [00:00<00:00, 3728.03it/s]
INFO:root:The number of level_3 predicted mapping is 2
INFO:root:Skip the level_4 mapping.
INFO:root:Skip the level_5 mapping.
INFO:root:Skip the level_6 mappping.
INFO:root:TOTAL NUMBER OF MAPPINGS BETWEEN OBJECT PROPERTIES IS 2
INFO:root:========================================================
INFO:root:MAP DATATYPE PROPERTIES
INFO:root:Retrieving Synsets by WordNet..

In [45]:
preds = maponto.load_alignments(output_url, None)
maponto.evaluate(preds, refs_url)

Total number of None is 413
Total number of references is 63
Total correctly predicted alignments is 57
Total number of predicted is 413
Precision is 0.13801452784503632
Recall is 0.9047619047619048
F1-Measure is 0.23949579831932774


In [63]:
aligns = maponto.load_alignments(output_url, None)

Total number of None is 413


In [64]:
aligns.iloc[1]

source        http://codata.jp/OML-MaterialInformation#Amoun...
target        http://emmo.info/emmo/middle/isq#EMMO_8159c26a...
relation                                                      =
confidence                                                  1.0
Name: 1, dtype: object

In [65]:
refs = maponto.load_alignments(refs_url, None)
refs

Total number of None is 63


,source,target,relation,confidence
0,http://codata.jp/OML-MaterialInformation#Energy,http://emmo.info/emmo/middle/isq#EMMO_31ec09ba...,=,0.9306
1,http://codata.jp/OML-MaterialInformation#Illum...,http://emmo.info/emmo/middle/isq#EMMO_b51fbd00...,=,0.9306
2,http://codata.jp/OML-MaterialInformation#Density,http://emmo.info/emmo/middle/isq#EMMO_06448f64...,=,0.9306
3,http://codata.jp/OML-MaterialInformation#Lumin...,http://emmo.info/emmo/middle/isq#EMMO_e2ee1c98...,=,0.9306
4,http://codata.jp/OML-MaterialInformation#Magne...,http://emmo.info/emmo/middle/isq#EMMO_3b931698...,=,0.9306
...,...,...,...,...
58,http://codata.jp/OML-MaterialInformation#Elect...,http://emmo.info/emmo/middle/isq#EMMO_e150fa8d...,=,1.0
59,http://codata.jp/OML-MaterialInformation#SI_units,http://emmo.info/emmo/middle/siunits#EMMO_feb0...,=,1.0
60,http://codata.jp/OML-MaterialInformation#CGS_u...,http://emmo.info/emmo/middle/units-extension#E...,=,1.0
61,http://codata.jp/OML-MaterialInformation#WaveN...,http://emmo.info/emmo/middle/isq#EMMO_d859588d...,=,1.0


In [58]:
target_graph = Graph()
stack_urls = []
visited_urls = []
stack_urls.append(target_url)
maptonto.parse_owl_withImports(target_graph, stack_urls, visited_urls)
len(target_graph)

../data/MSE-Benchmark/testCases/thirdTestCase/EMMO.owl
http://emmo.info/emmo/1.0.0-alpha2/middle/properties
http://emmo.info/emmo/1.0.0-alpha2/middle/reductionistic
http://emmo.info/emmo/1.0.0-alpha2/middle/materials
http://emmo.info/emmo/1.0.0-alpha2/middle/isq
http://emmo.info/emmo/1.0.0-alpha2/middle/metrology
http://emmo.info/emmo/1.0.0-alpha2/middle/math
http://emmo.info/emmo/1.0.0-alpha2/middle/manufacturing
http://emmo.info/emmo/1.0.0-alpha2/top
http://emmo.info/emmo/1.0.0-alpha2/top/physical
http://emmo.info/emmo/1.0.0-alpha2/top/mereotopology
http://emmo.info/emmo/1.0.0-alpha2/top/annotations
http://emmo.info/emmo/1.0.0-alpha2/middle/siunits
http://emmo.info/emmo/1.0.0-alpha2/middle/units-extension
http://emmo.info/emmo/1.0.0-alpha2/middle/semiotics
http://emmo.info/emmo/1.0.0-alpha2/middle/holistic
http://emmo.info/emmo/1.0.0-alpha2/middle/models
http://emmo.info/emmo/1.0.0-alpha2/middle/physicalistic
http://emmo.info/emmo/1.0.0-alpha2/middle/perceptual


4644

# Retrieve skos:prefLabel

In [59]:
maponto.extract_label_uris(target_graph)

,label,uri
0,Holistic,http://emmo.info/emmo/middle/holistic#EMMO_027...
1,Process,http://emmo.info/emmo/middle/holistic#EMMO_43e...
2,Participant,http://emmo.info/emmo/middle/holistic#EMMO_498...
3,TimeDimension,http://emmo.info/emmo/middle/isq#EMMO_02e894c3...
4,Velocity,http://emmo.info/emmo/middle/isq#EMMO_0329f1f5...
...,...,...
445,EMMO,http://emmo.info/emmo/top/mereotopology#EMMO_8...
446,Item,http://emmo.info/emmo/top/mereotopology#EMMO_e...
447,Elementary,http://emmo.info/emmo/top/physical#EMMO_0f795e...
448,Void,http://emmo.info/emmo/top/physical#EMMO_29072e...


In [45]:
#target_url = "../data/anatomy_track/anatomy_track-default/mouse-human-suite/target.rdf"
target_url = "../data/MSE-Benchmark/testCases/thirdTestCase/EMMO.owl"

In [46]:
target_graph = Graph()
stack_urls = []
stack_urls.append(target_url)
visited_urls = []
maponto.parse_owl_withImports(target_graph, stack_urls, visited_urls)

../data/MSE-Benchmark/testCases/thirdTestCase/EMMO.owl
http://emmo.info/emmo/1.0.0-alpha2/middle/semiotics
http://emmo.info/emmo/1.0.0-alpha2/middle/manufacturing
http://emmo.info/emmo/1.0.0-alpha2/top
http://emmo.info/emmo/1.0.0-alpha2/top/physical
http://emmo.info/emmo/1.0.0-alpha2/top/mereotopology
http://emmo.info/emmo/1.0.0-alpha2/top/annotations
http://emmo.info/emmo/1.0.0-alpha2/middle/properties
http://emmo.info/emmo/1.0.0-alpha2/middle/isq
http://emmo.info/emmo/1.0.0-alpha2/middle/metrology
http://emmo.info/emmo/1.0.0-alpha2/middle/math
http://emmo.info/emmo/1.0.0-alpha2/middle/models
http://emmo.info/emmo/1.0.0-alpha2/middle/reductionistic
http://emmo.info/emmo/1.0.0-alpha2/middle/materials
http://emmo.info/emmo/1.0.0-alpha2/middle/physicalistic
http://emmo.info/emmo/1.0.0-alpha2/middle/holistic
http://emmo.info/emmo/1.0.0-alpha2/middle/perceptual
http://emmo.info/emmo/1.0.0-alpha2/middle/siunits
http://emmo.info/emmo/1.0.0-alpha2/middle/units-extension


In [47]:
len(target_graph)

4644

In [48]:
from rdflib.namespace import Namespace, NamespaceManager
SKOS_NS = Namespace("http://www.w3.org/2004/02/skos/core#")
target_graph.namespace_manager.bind("skos", SKOS_NS)
    
def_query = '''SELECT DISTINCT ?a ?b
                   WHERE {
                   {?a a owl:Class .
                   ?a rdfs:label ?b .
                   }
                   UNION
                   {?a a owl:Class .
                   ?a skos:prefLabel ?b .
                   }
                   
                   FILTER(!isBlank(?a))}
                   
                   ORDER BY ?a
            '''
    
q_res = target_graph.query(def_query)

labels = []
uris = []
for res in q_res:
    if len(str(res[1])) > 0:
        labels.append(str(res[1]))
        uris.append(str(res[0]))

In [52]:
labels[:5]

['Holistic', 'Process', 'Participant', 'TimeDimension', 'Velocity']

# Anatomy

In [47]:
source_url = "../data/anatomy_track/anatomy_track-default/mouse-human-suite/source.rdf"
target_url = "../data/anatomy_track/anatomy_track-default/mouse-human-suite/target.rdf"
output_url = '../data/results/mouse_human_alignments.rdf'
refs_url = "../data/anatomy_track/anatomy_track-default/mouse-human-suite/reference.rdf"

In [48]:
mapper.align(source_url, target_url, output_url)

INFO:root:Python ensemble mapper info: map ../data/anatomy_track/anatomy_track-default/mouse-human-suite/source.rdf to ../data/anatomy_track/anatomy_track-default/mouse-human-suite/target.rdf


../data/anatomy_track/anatomy_track-default/mouse-human-suite/source.rdf


INFO:root:Read source with 15958 triples.


../data/anatomy_track/anatomy_track-default/mouse-human-suite/target.rdf


INFO:root:Read target with 35354 triples.
INFO:root:MAP CONCEPTS
INFO:root:The number of level_1 predicted mapping is 951.
INFO:root:Retrieving Synsets by WordNet...
INFO:root:The number of level_2 predicted mapping is 32
INFO:root:Skip the level_3 mapping.
INFO:root:Skip the level_4 mapping.
INFO:root:Skip the level_5 mapping.
INFO:root:Matching Label Embeddings by Optimal Transport...
INFO:root:Computing the Ground Embedding Costs between the Source and Target Points...
INFO:root:The shape of the cost matrix is (1757, 2315)
INFO:root:Computing Optimal Transport Plan...
INFO:root:Computing Wasserstein distance by the Sinkhorn algorithm...
INFO:root:Making Mappings from a Pairwise OT Plan Matrix by Mutual NN...
INFO:root:The number of level_6 predicted mapping is 920
INFO:root:TOTAL NUMBER OF MAPPINGS BETWEEN CONCPETS IS 1903
INFO:root:=================================================
INFO:root:MAP OBJECT PROPERTIES
INFO:root:The number of level_1 predicted mapping is 2.
INFO:root:Retr

In [49]:
preds = maponto.load_alignments(output_url, None)
maponto.evaluate(preds, refs_url)

Total number of None is 1905
Total number of references is 1516
Total correctly predicted alignments is 1230
Total number of predicted is 1905
Precision is 0.6456692913385826
Recall is 0.8113456464379947
F1-Measure is 0.7190879859690149


# Conference

In [12]:
source_url = "../data/conference/conference-v1/cmt-conference/source.rdf"
target_url = "../data/conference/conference-v1/cmt-conference/target.rdf"
output_url = '../data/results/cmt_conference_alignments.rdf'
refs_url = "../data/conference/conference-v1/cmt-conference/reference.rdf"
mapper.align(source_url, target_url, output_url)
preds = maponto.load_alignments(output_url, None)
maponto.evaluate(preds, refs_url)

INFO:root:Python ensemble mapper info: map ../data/conference/conference-v1/cmt-conference/source.rdf to ../data/conference/conference-v1/cmt-conference/target.rdf
INFO:root:Read source with 412 triples.


../data/conference/conference-v1/cmt-conference/source.rdf
../data/conference/conference-v1/cmt-conference/target.rdf


INFO:root:Read target with 639 triples.
INFO:root:MAP CONCEPTS
INFO:root:The number of level_1 predicted mapping is 6.
INFO:root:Retrieving Synsets by WordNet...
INFO:root:The number of level_2 predicted mapping is 1
INFO:root:Compute the related words for each source concept....
22it [00:00, 53.21it/s]
INFO:root:Total number of source concepts computed with relatd words is 22
INFO:root:Compute the related words for each target concept....
52it [00:01, 47.25it/s]
INFO:root:Total number of target concepts computed with related words is 52
100%|██████████| 22/22 [00:00<00:00, 3021.54it/s]
INFO:root:The number of level_3 predicted mapping is 7
INFO:root:Compute Pairwise Wasserstein Distances...
100%|██████████| 15/15 [00:00<00:00, 17.66it/s]
INFO:root:The number of level_4 predicted mapping is 1
INFO:root:Skip level_5 mapping.
INFO:root:Matching Label Embeddings by Optimal Transport...
INFO:root:Computing the Ground Embedding Costs between the Source and Target Points...
INFO:root:The sha

48it [00:01, 46.04it/s]
INFO:root:Total number of source concepts computed with relatd words is 48
INFO:root:Compute the related words for each target concept....
45it [00:00, 56.02it/s]
INFO:root:Total number of target concepts computed with related words is 45
100%|██████████| 48/48 [00:00<00:00, 3770.94it/s]
INFO:root:The number of level_3 predicted mapping is 6
INFO:root:Skip the level_4 mapping.
INFO:root:Skip the level_5 mapping.
INFO:root:Skip the level_6 mappping.
INFO:root:TOTAL NUMBER OF MAPPINGS BETWEEN OBJECT PROPERTIES IS 7
INFO:root:========================================================
INFO:root:MAP DATATYPE PROPERTIES
INFO:root:The number of level_1 predicted mapping is 2.
INFO:root:Retrieving Synsets by WordNet...
INFO:root:Compute the related words for each source concept....
8it [00:00, 56.19it/s]
INFO:root:Total number of source concepts computed with relatd words is 8
INFO:root:Compute the related words for each target concept....
16it [00:00, 47.04it/s]
INFO:roo

Total number of None is 39
Total number of references is 15
Total correctly predicted alignments is 10
Total number of predicted is 39
Precision is 0.2564102564102564
Recall is 0.6666666666666666
F1-Measure is 0.37037037037037035


In [14]:
source_graph = Graph()
stack_urls = []
visited_urls = []
stack_urls.append(source_url)
maponto.parse_owl_withImports(source_graph, stack_urls, visited_urls)
#len(source_graph)
target_graph = Graph()
stack_urls = []
visited_urls = []
stack_urls.append(target_url)
maponto.parse_owl_withImports(target_graph, stack_urls, visited_urls)
len(source_graph), len(target_graph)

../data/conference/conference-v1/cmt-conference/source.rdf
../data/conference/conference-v1/cmt-conference/target.rdf


(412, 639)

In [16]:
source_clndLabel_uris = maponto.clean_labels(maponto.extract_label_uris(source_graph))
source_clndLabel_uris

,label,uri,clndLabel
0,Acceptance,http://cmt#Acceptance,acceptance
1,Administrator,http://cmt#Administrator,administrator
2,AssociatedChair,http://cmt#AssociatedChair,associated chair
3,Author,http://cmt#Author,author
4,AuthorNotReviewer,http://cmt#AuthorNotReviewer,author reviewer
5,Bid,http://cmt#Bid,bid
6,Chairman,http://cmt#Chairman,chairman
7,Co-author,http://cmt#Co-author,co author
8,Conference,http://cmt#Conference,conference
9,ConferenceChair,http://cmt#ConferenceChair,conference chair


In [17]:
target_clndLabel_uris = maponto.clean_labels(maponto.extract_label_uris(target_graph))
target_clndLabel_uris

,label,uri,clndLabel
0,Abstract,http://conference#Abstract,abstract
1,Accepted_contribution,http://conference#Accepted_contribution,accepted contribution
2,Active_conference_participant,http://conference#Active_conference_participant,active conference participant
3,Call_for_paper,http://conference#Call_for_paper,call paper
4,Call_for_participation,http://conference#Call_for_participation,call participation
5,Camera_ready_contribution,http://conference#Camera_ready_contribution,camera ready contribution
6,Chair,http://conference#Chair,chair
7,Co-chair,http://conference#Co-chair,co chair
8,Committee,http://conference#Committee,committee
9,Committee_member,http://conference#Committee_member,committee member


In [19]:
refs_df = maponto.load_alignments(refs_url, None)
refs_df

Total number of None is 15


,source,target,relation,confidence
0,http://cmt#Conference,http://conference#Conference_volume,=,1.0
1,http://cmt#Preference,http://conference#Review_preference,=,1.0
2,http://cmt#Author,http://conference#Regular_author,=,1.0
3,http://cmt#Person,http://conference#Person,=,1.0
4,http://cmt#email,http://conference#has_an_email,=,1.0
5,http://cmt#Co-author,http://conference#Contribution_co-author,=,1.0
6,http://cmt#PaperAbstract,http://conference#Abstract,=,1.0
7,http://cmt#Document,http://conference#Conference_document,=,1.0
8,http://cmt#Review,http://conference#Review,=,1.0
9,http://cmt#Conference,http://conference#Conference,=,1.0


In [20]:
preds_df = maponto.load_alignments(output_url, None)
preds_df

Total number of None is 39


,source,target,relation,confidence
0,http://cmt#Conference,http://conference#Conference,=,1.0
1,http://cmt#Paper,http://conference#Paper,=,1.0
2,http://cmt#Person,http://conference#Person,=,1.0
3,http://cmt#ProgramCommittee,http://conference#Program_committee,=,1.0
4,http://cmt#Review,http://conference#Review,=,1.0
5,http://cmt#Reviewer,http://conference#Reviewer,=,1.0
6,http://cmt#Chairman,http://conference#Chair,=,1.0
7,http://cmt#Author,http://conference#Regular_author,=,1.0
8,http://cmt#Co-author,http://conference#Contribution_co-author,=,1.0
9,http://cmt#ConferenceChair,http://conference#Co-chair,=,1.0


In [22]:
relation_graph = maponto.build_relation_graph(source_graph)
relation_graph

INFO:root:Building a relation graph from the given RDF triple graph...


,subject,predicate,object
0,http://cmt#Paper,http://cmt#readByReviewer,http://cmt#Reviewer
1,http://cmt#ExternalReviewer,http://cmt#assignedByReviewer,http://cmt#Reviewer
2,http://cmt#Paper,http://cmt#readByMeta-Reviewer,http://cmt#Meta-Reviewer
3,http://cmt#Paper,http://cmt#hasDecision,http://cmt#Decision
4,http://cmt#Paper,http://cmt#hasDecision,http://cmt#Decision
5,http://cmt#AuthorNotReviewer,rdfs:subClassOf,http://cmt#Author
6,http://cmt#Administrator,rdfs:subClassOf,http://cmt#User
7,http://cmt#Meta-Reviewer,rdfs:subClassOf,http://cmt#Reviewer
8,http://cmt#ExternalReviewer,rdfs:subClassOf,http://cmt#Person
9,http://cmt#Rejection,rdfs:subClassOf,http://cmt#Decision


# For a URI in an RDF graph, extract all related classes and properties

In [323]:
importlib.reload(maponto)

<module 'OTMapOnto' from '/home/ubuntu/django/otmaponto/match/ontology_mapping/src/OTMapOnto.py'>

In [324]:
maponto.query_superClasses("http://cmt#Review", source_graph)

(['rdfs:subClassOf'], ['http://cmt#Document'], ['child'])

In [325]:
maponto.query_subClasses("http://cmt#User", source_graph)

(['superClassOf', 'superClassOf', 'superClassOf'],
 ['http://cmt#Author', 'http://cmt#Reviewer', 'http://cmt#Administrator'],
 ['parent', 'parent', 'parent'])

In [326]:
maponto.query_relations_someValuesFrom("http://conference#Review_expertise", target_graph)

(['http://conference#has_a_submitted_contribution',
  'http://conference#belongs_to_reviewers'],
 ['http://conference#Submitted_contribution', 'http://conference#Reviewer'],
 ['someValuesFrom', 'someValuesFrom'])

In [327]:
maponto.query_relations_allValuesFrom("http://conference#Review", target_graph)

(['http://conference#has_authors'],
 ['http://conference#Reviewer'],
 ['allValuesFrom'])

In [328]:
maponto.query_relations_minCardinality("http://cmt#Paper", source_graph)

(['http://cmt#readByReviewer', 'http://cmt#hasDecision'],
 [1, 0],
 ['owl:minCardinality', 'owl:minCardinality'])

In [329]:
maponto.query_relations_maxCardinality("http://cmt#Paper", source_graph)

(['http://cmt#readByMeta-Reviewer', 'http://cmt#hasDecision'],
 [1, 1],
 ['owl:maxCardinality', 'owl:maxCardinality'])

In [330]:
maponto.query_relations_asDomains("http://cmt#Paper", source_graph)

(['http://cmt#readByReviewer',
  'http://cmt#readByMeta-Reviewer',
  'http://cmt#hasDecision',
  'http://cmt#hasDecision',
  'http://cmt#acceptedBy',
  'http://cmt#readByMeta-Reviewer',
  'http://cmt#hasDecision',
  'http://cmt#assignedTo',
  'http://cmt#rejectedBy',
  'http://cmt#hasSubjectArea',
  'http://cmt#readByReviewer',
  'http://cmt#hasCo-author',
  'http://cmt#hasAuthor',
  'http://cmt#hasBid'],
 ['http://cmt#Reviewer',
  'http://cmt#Meta-Reviewer',
  'http://cmt#Decision',
  'http://cmt#Decision',
  'http://cmt#Administrator',
  'http://cmt#Meta-Reviewer',
  'http://cmt#Decision',
  'http://cmt#Reviewer',
  'http://cmt#Administrator',
  'http://cmt#SubjectArea',
  'http://cmt#Reviewer',
  'http://cmt#Co-author',
  'http://cmt#Author',
  'http://cmt#Bid'],
 ['domain',
  'domain',
  'domain',
  'domain',
  'domain',
  'domain',
  'domain',
  'domain',
  'domain',
  'domain',
  'domain',
  'domain',
  'domain',
  'domain'])

In [331]:
maponto.query_datatype_properties("http://conference#Person", target_graph)

(['http://conference#has_the_last_name',
  'http://conference#has_the_first_name',
  'http://conference#has_an_email'],
 ['http://www.w3.org/2001/XMLSchema#string',
  'http://www.w3.org/2001/XMLSchema#string',
  'http://www.w3.org/2001/XMLSchema#string'],
 ['datatype domain', 'datatype domain', 'datatype domain'])

In [332]:
maponto.query_relations_asRanges("http://cmt#Paper", source_graph)

(['http://cmt#hasBeenAssigned',
  'http://cmt#co-writePaper',
  'http://cmt#submitPaper',
  'http://cmt#rejectPaper',
  'http://cmt#acceptPaper',
  'http://cmt#readPaper',
  'http://cmt#writePaper'],
 ['http://cmt#Reviewer',
  'http://cmt#Co-author',
  'http://cmt#Author',
  'http://cmt#Administrator',
  'http://cmt#Administrator',
  'http://cmt#Reviewer',
  'http://cmt#Author'],
 ['range', 'range', 'range', 'range', 'range', 'range', 'range'])

In [333]:
maponto.query_comments("http://cmt#Meta-Reviewer", source_graph)

(['rdfs:comment'],
 ['A special type of Reviewer.  There can be any number of Reviewers for a given paper, but only one Meta-Reviewer.  The Meta-Reviewer can go over all the reviews submitted for the paper and submit their own review.  Questions for the Meta-Reviewer can be different from those for a normal Reviewer.  Their role is to ensure that the reviews are good and consistent. The decision to use Meta-Reviewers is optional, and must be set before paper assignment occurs.'],
 ['comment'])

In [334]:
maponto.query_equivalent_classes("http://cmt#Chairman", source_graph)

(['owl:equivalentClass', 'owl:equivalentClass', 'owl:equivalentClass'],
 ['http://cmt#AssociatedChair',
  'http://cmt#ConferenceChair',
  'http://cmt#ProgramCommitteeChair'],
 ['equivalent', 'equivalent', 'equivalent'])

In [335]:
maponto.query_disjoint_classes("http://cmt#Preference", source_graph)

(['owl:disjointWith',
  'owl:disjointWith',
  'owl:disjointWith',
  'owl:disjointWith',
  'owl:disjointWith',
  'owl:disjointWith'],
 ['http://cmt#Bid',
  'http://cmt#ProgramCommittee',
  'http://cmt#Conference',
  'http://cmt#Decision',
  'http://cmt#Person',
  'http://cmt#Document'],
 ['disjoint', 'disjoint', 'disjoint', 'disjoint', 'disjoint', 'disjoint'])

In [336]:
maponto.query_related_information("http://cmt#SubjectArea", source_graph)

,relation,related,what
0,http://cmt#hasSubjectArea,http://cmt#Paper,range


In [337]:
maponto.query_related_information("http://conference#Topic", target_graph)

,relation,related,what
0,http://conference#belongs_to_a_review_reference,http://conference#Review_preference,domain
1,http://conference#is_a_topis_of_conference_parts,http://conference#Conference_part,domain
2,http://conference#has_a_name,http://www.w3.org/2001/XMLSchema#string,datatype domain
3,http://conference#has_a_track-workshop-tutoria...,http://conference#Conference_part,range


# List Variables and Sizes

In [10]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in globals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

                         preds: 11.1 KiB
                             _:  2.2 KiB
                            _7:  2.2 KiB
                         Graph:  1.2 KiB
                          _i10:  588.0 B
                            _i:  587.0 B
                           _i9:  587.0 B
                           _i3:  465.0 B
                           _i5:  465.0 B
                    local_vars:  408.0 B


# =================================
## Other Tracks
# =================================

# IIMB Benchmark

In [10]:
source_url = "../data/iimb/v1/072/source.rdf"
target_url = "../data/iimb/v1/072/target.rdf"
output_url = '../data/results/iimb_source_target_072_alignment.rdf'
refs_url = "../data/iimb/v1/072/reference.rdf"

In [11]:
mapper.align(source_url, target_url, output_url)

INFO:root:Python ensemble mapper info: map ../data/iimb/v1/072/source.rdf to ../data/iimb/v1/072/target.rdf


../data/iimb/v1/072/source.rdf


INFO:root:Read source with 2617 triples.


../data/iimb/v1/072/target.rdf


INFO:root:Read target with 5874 triples.
INFO:root:MAP CONCEPTS
INFO:root:The number of level_1 predicted mapping is 29.
INFO:root:Retrieving Synsets by WordNet...
INFO:root:Compute the related words for each source concept....
0it [00:00, ?it/s]
INFO:root:Total number of source concepts computed with relatd words is 0
INFO:root:Compute the related words for each target concept....
0it [00:00, ?it/s]
INFO:root:Total number of target concepts computed with related words is 0
INFO:root:Compute Pairwise Wasserstein Distances...
INFO:root:Skip level_5 mapping.
INFO:root:TOTAL NUMBER OF MAPPINGS BETWEEN CONCPETS IS 29
INFO:root:=================================================
INFO:root:MAP OBJECT PROPERTIES
INFO:root:The number of level_1 predicted mapping is 36.
INFO:root:Retrieving Synsets by WordNet...
INFO:root:Compute the related words for each source concept....
0it [00:00, ?it/s]
INFO:root:Total number of source concepts computed with relatd words is 0
INFO:root:Compute the related 

In [12]:
preds = maponto.load_alignments(output_url, None)
maponto.evaluate(preds, refs_url)

Total number of None is 78
Total number of references is 169
Total correctly predicted alignments is 0


ZeroDivisionError: float division by zero

# Knowledge Graph

In [51]:
source_url = "../data/knowledgegraph/commonkg/nell-dbpedia/source.rdf"
target_url = "../data/knowledgegraph/commonkg/nell-dbpedia/target.rdf"
output_url = '../data/results/knowledgegraph_commonkg_alignment.rdf'
refs_url = "../data/knowledgegraph/commonkg/nell-dbpedia/reference.rdf"
#source_url = "../data/knowledgegraph/small-test/test-small/source.rdf"
#target_url = "../data/knowledgegraph/small-test/test-small/target.rdf"
#output_url = '../data/results/knowledgegraph_small_test_alignment.rdf'
#refs_url = "../data/knowledgegraph/small-test/test-small/reference.rdf"

In [52]:
mapper.align(source_url, target_url, output_url)

INFO:root:Python ensemble mapper info: map ../data/knowledgegraph/commonkg/nell-dbpedia/source.rdf to ../data/knowledgegraph/commonkg/nell-dbpedia/target.rdf


../data/knowledgegraph/commonkg/nell-dbpedia/source.rdf


INFO:root:Read source with 2016539 triples.


../data/knowledgegraph/commonkg/nell-dbpedia/target.rdf


INFO:root:Read target with 1342254 triples.
INFO:root:MAP CONCEPTS
INFO:root:The number of level_1 predicted mapping is 78.
INFO:root:Retrieving Synsets by WordNet...
INFO:root:The number of level_2 predicted mapping is 3
INFO:root:Compute the related words for each source concept....
14it [00:00, 63.56it/s]

...

OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)


56it [00:00, 56.26it/s]
INFO:root:Total number of target concepts computed with related words is 56


OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)


100%|██████████| 53/53 [00:00<00:00, 5288.03it/s]
INFO:root:The number of level_3 predicted mapping is 3
INFO:root:Compute Pairwise Wasserstein Distances...
100%|██████████| 50/50 [00:02<00:00, 24.18it/s]
INFO:root:The number of level_4 predicted mapping is 1
INFO:root:Skip level_5 mapping.
INFO:root:Matching Label Embeddings by Optimal Transport...
INFO:root:Computing the Ground Embedding Costs between the Source and Target Points...
INFO:root:The shape of the cost matrix is (49, 52)
INFO:root:Computing Optimal Transport Plan...
INFO:root:Computing Wasserstein distance by the Sinkhorn algorithm...
INFO:root:The Sinkhorn got warnings. Computing Wasserstein distance by the EMD algorithm...
INFO:root:Making Mappings from a Pairwise OT Plan Matrix by Mutual NN...
INFO:root:The number of level_6 predicted mapping is 38
INFO:root:TOTAL NUMBER OF MAPPINGS BETWEEN CONCPETS IS 123
INFO:root:=================================================
INFO:root:MAP OBJECT PROPERTIES
INFO:root:Retrieving S

0it [00:00, ?it/s]
INFO:root:Total number of source concepts computed with relatd words is 0
INFO:root:Compute the related words for each target concept....
0it [00:00, ?it/s]
INFO:root:Total number of target concepts computed with related words is 0
INFO:root:Skip the level_4 mapping.
INFO:root:Skip the level_5 mapping.
INFO:root:Skip the level_6 mappping.
INFO:root:TOTAL NUMBER OF MAPPINGS BETWEEN OBJECT PROPERTIES IS 0
INFO:root:========================================================
INFO:root:MAP DATATYPE PROPERTIES
INFO:root:Retrieving Synsets by WordNet...
INFO:root:Compute the related words for each source concept....
0it [00:00, ?it/s]
INFO:root:Total number of source concepts computed with relatd words is 0
INFO:root:Compute the related words for each target concept....
0it [00:00, ?it/s]
INFO:root:Total number of target concepts computed with related words is 0
INFO:root:Skip the level_4 mapping.
INFO:root:Skip the level_5 mapping.
INFO:root:Skip the level_6 mappping.
INFO:r

In [53]:
preds = maponto.load_alignments(output_url, None)
maponto.evaluate(preds, refs_url)

Total number of None is 123
Total number of references is 129
Total correctly predicted alignments is 109
Total number of predicted is 123
Precision is 0.8861788617886179
Recall is 0.8449612403100775
F1-Measure is 0.8650793650793651


In [4]:
source_url = "../data/knowledgegraph/v3/ontologies/starwars.rdf"
target_url = "../data/knowledgegraph/v3/ontologies/swg.rdf"
output_url = '../data/results/knowledgegraph_starwars_swg_alignment.rdf'
refs_url = "../data/knowledgegraph/v3/references/starwars-swg.rdf"

In [5]:
mapper.align(source_url, target_url, output_url)

INFO:root:Python ensemble mapper info: map ../data/knowledgegraph/v3/ontologies/starwars.rdf to ../data/knowledgegraph/v3/ontologies/swg.rdf
INFO:root:Read source with 8246033 triples.
INFO:root:Read target with 254354 triples.
INFO:root:MAP CONCEPTS
INFO:root:The number of level_1 predicted mapping is 12.
INFO:root:Retrieving Synsets by WordNet...
INFO:root:The number of level_2 predicted mapping is 4
INFO:root:Compute the related words for each source concept....
9it [00:00, 83.70it/s]

OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.quer

27it [00:00, 81.43it/s]

OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.quer

45it [00:00, 78.08it/s]

...

28it [00:00, 85.36it/s]

OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.quer

51it [00:00, 86.42it/s]
INFO:root:Total number of target concepts computed with related words is 51


OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.quer

100%|██████████| 255/255 [00:00<00:00, 8959.94it/s]
INFO:root:The number of level_3 predicted mapping is 13
INFO:root:Compute Pairwise Wasserstein Distances...
 17%|█▋        | 40/242 [00:01<00:07, 25.56it/s]/Users/yan/anaconda3/lib/python3.7/site-packages/ot/lp/__init__.py:421: UserWarning: Problem infeasible. Check that a and b are in the simplex
  check_result(result_code)
100%|██████████| 242/242 [00:07<00:00, 34.19it/s]
INFO:root:The number of level_4 predicted mapping is 3
INFO:root:Skip level_5 mapping.
INFO:root:Matching Label Embeddings by Optimal Transport...
INFO:root:Computing the Ground Embedding Costs between the Source and Target Points...
INFO:root:The shape of the cost matrix is (239, 35)
INFO:root:Computing Optimal Transport Plan...
INFO:root:Computing Wasserstein distance by the Sinkhorn algorithm...
INFO:root:The Sinkhorn got warnings. Computing Wasserstein distance by the EMD algorithm...
INFO:root:Making Mappings from a Pairwise OT Plan Matrix by Mutual NN...
INFO

In [6]:
preds = maponto.load_alignments(output_url, None)
maponto.evaluate(preds, refs_url)

Total number of None is 67
Total number of references is 1121
Total correctly predicted alignments is 6
Total number of predicted is 67
Precision is 0.08955223880597014
Recall is 0.0053523639607493305
F1-Measure is 0.010101010101010102


# Biodiv

In [54]:
source_url = "../data/biodiv/2018/envo-sweet/source.rdf"
target_url = "../data/biodiv/2018/envo-sweet/target.rdf"
output_url = '../data/results/biodiv_2018_envo_sweet_alignment.rdf'
refs_url = "../data/biodiv/2018/envo-sweet/reference.rdf"

In [55]:
mapper.align(source_url, target_url, output_url)

INFO:root:Python ensemble mapper info: map ../data/biodiv/2018/envo-sweet/source.rdf to ../data/biodiv/2018/envo-sweet/target.rdf


../data/biodiv/2018/envo-sweet/source.rdf


INFO:root:Read source with 83591 triples.


../data/biodiv/2018/envo-sweet/target.rdf


INFO:root:Read target with 27799 triples.
INFO:root:MAP CONCEPTS
INFO:root:The number of level_1 predicted mapping is 469.
INFO:root:Retrieving Synsets by WordNet...
INFO:root:The number of level_2 predicted mapping is 434
INFO:root:Skip the level_3 mapping.
INFO:root:Skip the level_4 mapping.
INFO:root:Skip the level_5 mapping.
INFO:root:Matching Label Embeddings by Optimal Transport...
INFO:root:Computing the Ground Embedding Costs between the Source and Target Points...
INFO:root:The shape of the cost matrix is (6349, 3808)
INFO:root:Computing Optimal Transport Plan...
INFO:root:Computing Wasserstein distance by the Sinkhorn algorithm...


...

OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)


21it [00:00, 59.86it/s]

OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)


32it [00:00, 59.74it/s]
INFO:root:Total number of target concepts computed with related words is 32
INFO:root:Skip the level_4 mapping.
INFO:root:Skip the level_5 mapping.
INFO:root:Skip the level_6 mappping.
INFO:root:TOTAL NUMBER OF MAPPINGS BETWEEN DATATYPE PROPERTIES IS 0
INFO:root:===========================================================


OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)
OTNeighborhood_TDA.get_relatedWords_count(): res=rdfgraph.query(q)


In [56]:
preds = maponto.load_alignments(output_url, None)
maponto.evaluate(preds, refs_url)

Total number of None is 4422
Total number of references is 666
Total correctly predicted alignments is 511
Total number of predicted is 4422
Precision is 0.11555857078245138
Recall is 0.7672672672672672
F1-Measure is 0.20086477987421386


In [57]:
source_url = "../data/biodiv/2018/flopo-pto/source.rdf"
target_url = "../data/biodiv/2018/flopo-pto/target.rdf"
output_url = '../data/results/biodiv_2018_flopo_pto_alignment.rdf'
refs_url = "../data/biodiv/2018/flopo-pto/reference.rdf"

In [58]:
mapper.align(source_url, target_url, output_url)

INFO:root:Python ensemble mapper info: map ../data/biodiv/2018/flopo-pto/source.rdf to ../data/biodiv/2018/flopo-pto/target.rdf


../data/biodiv/2018/flopo-pto/source.rdf
http://purl.obolibrary.org/obo/po.owl
http://purl.obolibrary.org/obo/po/imports/ro_import.owl
http://purl.obolibrary.org/obo/po/imports/ncbitaxon_import.owl


INFO:root:Read source with 567475 triples.


http://purl.obolibrary.org/obo/pato.owl
../data/biodiv/2018/flopo-pto/target.rdf


INFO:root:Read target with 24638 triples.
INFO:root:MAP CONCEPTS
INFO:root:The number of level_1 predicted mapping is 140.
INFO:root:Retrieving Synsets by WordNet...
INFO:root:The number of level_2 predicted mapping is 3
INFO:root:Skip the level_3 mapping.
INFO:root:Skip the level_4 mapping.
INFO:root:Skip the level_5 mapping.
INFO:root:Matching Label Embeddings by Optimal Transport...
INFO:root:Computing the Ground Embedding Costs between the Source and Target Points...
INFO:root:The shape of the cost matrix is (31389, 1338)
INFO:root:Computing Optimal Transport Plan...
INFO:root:Computing Wasserstein distance by the Sinkhorn algorithm...
INFO:root:Making Mappings from a Pairwise OT Plan Matrix by Mutual NN...
INFO:root:The number of level_6 predicted mapping is 1290
INFO:root:TOTAL NUMBER OF MAPPINGS BETWEEN CONCPETS IS 1433
INFO:root:=================================================
INFO:root:MAP OBJECT PROPERTIES
INFO:root:The number of level_1 predicted mapping is 1.
INFO:root:Ret

In [59]:
preds = maponto.load_alignments(output_url, None)
maponto.evaluate(preds, refs_url)

Total number of None is 1434
Total number of references is 245
Total correctly predicted alignments is 173
Total number of predicted is 1434
Precision is 0.1206415620641562
Recall is 0.7061224489795919
F1-Measure is 0.2060750446694461


# largebio

In [60]:
source_url = "../data/largebio/largebio-all_tasks_2016/small-fma-nci/source.rdf"
target_url = "../data/largebio/largebio-all_tasks_2016/small-fma-nci/target.rdf"
output_url = '../data/results/largebio_small_fma_nci_alignment.rdf'
refs_url = "../data/largebio/largebio-all_tasks_2016/small-fma-nci/reference.rdf"

In [378]:
source_graph = Graph().parse(source_url)
target_graph = Graph().parse(target_url)

In [379]:
slabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(source_graph))
tlabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(target_graph))
slabel_clnd_uris.shape, tlabel_clnd_uris.shape

((9142, 3), (17046, 3))

In [404]:
tlabel_clnd_uris = tlabel_clnd_uris[tlabel_clnd_uris.clndLabel != '']

In [405]:
sembs = []
tembs = []
for slabel in slabel_clnd_uris.clndLabel:
    sembs.append(maponto.average_embeddings(slabel, 300, embs_model))
for tlabel in tlabel_clnd_uris.clndLabel:
    tembs.append(maponto.average_embeddings(tlabel, 300, embs_model))

In [406]:
len(sembs), len(tembs)

(9142, 17044)

In [407]:
for i, emb in enumerate(tembs):
    try:
        if len(emb) != 300:
            print(emb)
    except:
        print(i, emb)

In [409]:
maponto.match_label_embeddings_OT(slabel_clnd_uris, tlabel_clnd_uris, embs_model, \
                        maponto.make_mappings_nn, None, None)

INFO:root:Matching Label Embeddings by Optimal Transport...
INFO:root:Computing the Ground Embedding Costs between the Source and Target Points...
INFO:root:The shape of the cost matrix is (9142, 17044)
INFO:root:Computing Optimal Transport Plan...
INFO:root:Computing Wasserstein distance by the Sinkhorn algorithm...


INFO:root:The Sinkhorn got warnings. Computing Wasserstein distance by the EMD algorithm...
INFO:root:Making Mappings from a Pairwise OT Plan Matrix by Mutual NN...


,source,source_label,target,target_label
0,http://bioontology.org/projects/ontologies/fma...,APUD cell,http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus...,APUD Cell
1,http://bioontology.org/projects/ontologies/fma...,Abdominopelvic region,http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus...,Rectosigmoid Region
2,http://bioontology.org/projects/ontologies/fma...,Abdomen,http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus...,abdomen
3,http://bioontology.org/projects/ontologies/fma...,Abdominopelvis,http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus...,Emperipolesis
4,http://bioontology.org/projects/ontologies/fma...,Pars abdominalis aortae,http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus...,Brainstem Neuroglial Tumor
...,...,...,...,...
8343,http://bioontology.org/projects/ontologies/fma...,Zygomaticus major,http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus...,Zygomaticus Major
8344,http://bioontology.org/projects/ontologies/fma...,Musculus zygomaticus major,http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus...,Mesothelioma Malignant Recurrent
8345,http://bioontology.org/projects/ontologies/fma...,Zygomaticus minor,http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus...,Zygomaticus Minor
8346,http://bioontology.org/projects/ontologies/fma...,Musculus zygomaticus minor,http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus...,Malignant Cutaneous Melanoma


In [415]:
importlib.reload(maponto)

<module 'OTMapOnto' from '/Users/yan/Google Drive/django/otmaponto/match/ontology_mapping/src/OTMapOnto.py'>

In [61]:
mapper.align(source_url, target_url, output_url)

INFO:root:Python ensemble mapper info: map ../data/largebio/largebio-all_tasks_2016/small-fma-nci/source.rdf to ../data/largebio/largebio-all_tasks_2016/small-fma-nci/target.rdf


../data/largebio/largebio-all_tasks_2016/small-fma-nci/source.rdf


INFO:root:Read source with 16919 triples.


../data/largebio/largebio-all_tasks_2016/small-fma-nci/target.rdf


INFO:root:Read target with 64857 triples.
INFO:root:MAP CONCEPTS
INFO:root:The number of level_1 predicted mapping is 3123.
INFO:root:Retrieving Synsets by WordNet...
INFO:root:The number of level_2 predicted mapping is 25
INFO:root:Skip the level_3 mapping.
INFO:root:Skip the level_4 mapping.
INFO:root:Skip the level_5 mapping.
INFO:root:Matching Label Embeddings by Optimal Transport...
INFO:root:Computing the Ground Embedding Costs between the Source and Target Points...
INFO:root:The shape of the cost matrix is (2899, 12134)
INFO:root:Computing Optimal Transport Plan...
INFO:root:Computing Wasserstein distance by the Sinkhorn algorithm...


INFO:root:The Sinkhorn got warnings. Computing Wasserstein distance by the EMD algorithm...
INFO:root:Making Mappings from a Pairwise OT Plan Matrix by Mutual NN...
INFO:root:The number of level_6 predicted mapping is 2781
INFO:root:TOTAL NUMBER OF MAPPINGS BETWEEN CONCPETS IS 5929
INFO:root:=================================================
INFO:root:MAP OBJECT PROPERTIES
INFO:root:Retrieving Synsets by WordNet...
INFO:root:Compute the related words for each source concept....
0it [00:00, ?it/s]
INFO:root:Total number of source concepts computed with relatd words is 0
INFO:root:Compute the related words for each target concept....
63it [00:01, 55.86it/s]
INFO:root:Total number of target concepts computed with related words is 63
INFO:root:Skip the level_4 mapping.
INFO:root:Skip the level_5 mapping.
INFO:root:Skip the level_6 mappping.
INFO:root:TOTAL NUMBER OF MAPPINGS BETWEEN OBJECT PROPERTIES IS 0
INFO:root:========================================================
INFO:root:MAP DATAT

In [62]:
preds = maponto.load_alignments(output_url, 'predicted')
maponto.evaluate(preds, refs_url)

Total number of predicted is 5929
Total number of references is 3024
Total correctly predicted alignments is 3068
Total number of predicted is 5929
Precision is 0.5174565694046214
Recall is 1.0145502645502646
F1-Measure is 0.6853568636211327


In [63]:
source_url = "../data/largebio/largebio-all_tasks_2016/small-fma-snomed/source.rdf"
target_url = "../data/largebio/largebio-all_tasks_2016/small-fma-snomed/target.rdf"
output_url = '../data/results/largebio_small_fma_snomed_alignment.rdf'
refs_url = "../data/largebio/largebio-all_tasks_2016/small-fma-snomed/reference.rdf"

In [64]:
mapper.align(source_url, target_url, output_url)

INFO:root:Python ensemble mapper info: map ../data/largebio/largebio-all_tasks_2016/small-fma-snomed/source.rdf to ../data/largebio/largebio-all_tasks_2016/small-fma-snomed/target.rdf


../data/largebio/largebio-all_tasks_2016/small-fma-snomed/source.rdf


INFO:root:Read source with 47699 triples.


../data/largebio/largebio-all_tasks_2016/small-fma-snomed/target.rdf


INFO:root:Read target with 108957 triples.
INFO:root:MAP CONCEPTS
INFO:root:The number of level_1 predicted mapping is 1604.
INFO:root:Retrieving Synsets by WordNet...
INFO:root:The number of level_2 predicted mapping is 70
INFO:root:Skip the level_3 mapping.
INFO:root:Skip the level_4 mapping.
INFO:root:Skip the level_5 mapping.
INFO:root:Matching Label Embeddings by Optimal Transport...
INFO:root:Computing the Ground Embedding Costs between the Source and Target Points...
INFO:root:The shape of the cost matrix is (23230, 11809)
INFO:root:Computing Optimal Transport Plan...
INFO:root:Computing Wasserstein distance by the Sinkhorn algorithm...


INFO:root:The Sinkhorn got warnings. Computing Wasserstein distance by the EMD algorithm...
INFO:root:Making Mappings from a Pairwise OT Plan Matrix by Mutual NN...
INFO:root:The number of level_6 predicted mapping is 11139
INFO:root:TOTAL NUMBER OF MAPPINGS BETWEEN CONCPETS IS 12813
INFO:root:=================================================
INFO:root:MAP OBJECT PROPERTIES
INFO:root:Retrieving Synsets by WordNet...
INFO:root:Compute the related words for each source concept....
0it [00:00, ?it/s]
INFO:root:Total number of source concepts computed with relatd words is 0
INFO:root:Compute the related words for each target concept....
18it [00:00, 54.64it/s]
INFO:root:Total number of target concepts computed with related words is 18
INFO:root:Skip the level_4 mapping.
INFO:root:Skip the level_5 mapping.
INFO:root:Skip the level_6 mappping.
INFO:root:TOTAL NUMBER OF MAPPINGS BETWEEN OBJECT PROPERTIES IS 0
INFO:root:========================================================
INFO:root:MAP DAT

In [65]:
preds = maponto.load_alignments(output_url, 'predicted')
maponto.evaluate(preds, refs_url)

Total number of predicted is 12813
Total number of references is 9008
Total correctly predicted alignments is 6197
Total number of predicted is 12813
Precision is 0.48364941855927573
Recall is 0.6879440497335702
F1-Measure is 0.5679849686082215


In [421]:
source_url = "../data/largebio/largebio-all_tasks_2016/small-snomed-nci/source.rdf"
target_url = "../data/largebio/largebio-all_tasks_2016/small-snomed-nci/target.rdf"
output_url = '../data/results/largebio_small_snomed_nci_alignment.rdf'
refs_url = "../data/largebio/largebio-all_tasks_2016/small-snomed-nci/reference.rdf"

In [ ]:
# Kernel died on running this
#mapper.align(source_url, target_url, output_url)

INFO:root:Python ensemble mapper info: map ../data/largebio/largebio-all_tasks_2016/small-snomed-nci/source.rdf to ../data/largebio/largebio-all_tasks_2016/small-snomed-nci/target.rdf
INFO:root:Read source with 1027068 triples.
INFO:root:Read target with 249276 triples.
INFO:root:MAP CONCEPTS
INFO:root:The number of level_1 predicted mapping is 15049.
INFO:root:Retrieving Synsets by WordNet...
INFO:root:The number of level_2 predicted mapping is 624
INFO:root:Skip the level_3 mapping.
INFO:root:Skip the level_4 mapping.
INFO:root:Skip the level_5 mapping.
INFO:root:Matching Label Embeddings by Optimal Transport...
INFO:root:Computing the Ground Embedding Costs between the Source and Target Points...
INFO:root:The shape of the cost matrix is (41031, 36011)
INFO:root:Computing Optimal Transport Plan...
INFO:root:Computing Wasserstein distance by the Sinkhorn algorithm...


In [ ]:
preds = maponto.load_alignments(output_url, 'predicted')
maponto.evaluate(preds, refs_url)

# Phenotype

In [66]:
source_url = "../data/phenotype/phenotype-doid-ordo-2017-bioportal/doid-ordo/source.rdf"
target_url = "../data/phenotype/phenotype-doid-ordo-2017-bioportal/doid-ordo/target.rdf"
output_url = '../data/results/phenotye_doid_ordo_2017_alignment.rdf'
refs_url = "../data/phenotype/phenotype-doid-ordo-2017-bioportal/doid-ordo/reference.rdf"

In [67]:
mapper.align(source_url, target_url, output_url)

INFO:root:Python ensemble mapper info: map ../data/phenotype/phenotype-doid-ordo-2017-bioportal/doid-ordo/source.rdf to ../data/phenotype/phenotype-doid-ordo-2017-bioportal/doid-ordo/target.rdf


../data/phenotype/phenotype-doid-ordo-2017-bioportal/doid-ordo/source.rdf
http://purl.obolibrary.org/obo/doid/obo/ext.owl


INFO:root:parse_owl_withImports: graph.prase(): an exception occured.
INFO:root:Read source with 203208 triples.


../data/phenotype/phenotype-doid-ordo-2017-bioportal/doid-ordo/target.rdf


INFO:root:Read target with 962199 triples.
INFO:root:MAP CONCEPTS
INFO:root:The number of level_1 predicted mapping is 1370.
INFO:root:Retrieving Synsets by WordNet...
INFO:root:The number of level_2 predicted mapping is 4
INFO:root:Skip the level_3 mapping.
INFO:root:Skip the level_4 mapping.
INFO:root:Skip the level_5 mapping.
INFO:root:Matching Label Embeddings by Optimal Transport...
INFO:root:Computing the Ground Embedding Costs between the Source and Target Points...
INFO:root:The shape of the cost matrix is (9594, 12231)
INFO:root:Computing Optimal Transport Plan...
INFO:root:Computing Wasserstein distance by the Sinkhorn algorithm...


INFO:root:The Sinkhorn got warnings. Computing Wasserstein distance by the EMD algorithm...
INFO:root:Making Mappings from a Pairwise OT Plan Matrix by Mutual NN...
INFO:root:The number of level_6 predicted mapping is 6336
INFO:root:TOTAL NUMBER OF MAPPINGS BETWEEN CONCPETS IS 7710
INFO:root:=================================================
INFO:root:MAP OBJECT PROPERTIES
INFO:root:Retrieving Synsets by WordNet...
INFO:root:Compute the related words for each source concept....
19it [00:00, 59.28it/s]
INFO:root:Total number of source concepts computed with relatd words is 19
INFO:root:Compute the related words for each target concept....
4it [00:00, 57.03it/s]
INFO:root:Total number of target concepts computed with related words is 4
100%|██████████| 19/19 [00:00<00:00, 33122.10it/s]
INFO:root:The number of level_3 predicted mapping is 0
INFO:root:Skip the level_4 mapping.
INFO:root:Skip the level_5 mapping.
INFO:root:Skip the level_6 mappping.
INFO:root:TOTAL NUMBER OF MAPPINGS BETWEEN

In [68]:
preds = maponto.load_alignments(output_url, 'predicted')
maponto.evaluate(preds, refs_url)

Total number of predicted is 7710
Total number of references is 1237
Total correctly predicted alignments is 1228
Total number of predicted is 7710
Precision is 0.15927367055771724
Recall is 0.9927243330638642
F1-Measure is 0.27450542081144513


In [9]:
source_graph = Graph().parse(source_url)
target_graph = Graph().parse(target_url)
slabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(source_graph))
tlabel_clnd_uris = maponto.clean_labels(maponto.extract_label_uris(target_graph))

In [10]:
maponto.evaluate(maponto.match_label_synonyms(slabel_clnd_uris, tlabel_clnd_uris, None), 
                 refs_url)

INFO:root:Retrieving Synsets by WordNet...


Total number of references is 1237
Total correctly predicted alignments is 1222
Total number of predicted is 1377
Precision is 0.8874364560639071
Recall is 0.9878738884397736
F1-Measure is 0.9349655700076511


In [40]:
source_url = "../data/phenotype/phenotype-hp-mp-2017-bioportal/hp-mp/source.rdf"
target_url = "../data/phenotype/phenotype-hp-mp-2017-bioportal/hp-mp/target.rdf"
output_url = '../data/results/phenotye_hp_mp_2017_alignment.rdf'
refs_url = "../data/phenotype/phenotype-hp-mp-2017-bioportal/hp-mp/reference.rdf"

In [41]:
mapper.align(source_url, target_url, output_url)

INFO:root:Python ensemble mapper info: map ../data/phenotype/phenotype-hp-mp-2017-bioportal/hp-mp/source.rdf to ../data/phenotype/phenotype-hp-mp-2017-bioportal/hp-mp/target.rdf


../data/phenotype/phenotype-hp-mp-2017-bioportal/hp-mp/source.rdf
http://purl.obolibrary.org/obo/upheno/imports/ro_extra.owl
http://purl.obolibrary.org/obo/upheno/imports/ro_import.owl
http://purl.obolibrary.org/obo/upheno/imports/pato_import.owl
http://purl.obolibrary.org/obo/upheno/imports/chebi_import.owl
http://purl.obolibrary.org/obo/upheno/imports/hsapdv_import.owl
http://purl.obolibrary.org/obo/upheno/imports/go_import.owl
http://purl.obolibrary.org/obo/upheno/imports/nbo_import.owl
http://purl.obolibrary.org/obo/upheno/imports/uberon_import.owl
http://purl.obolibrary.org/obo/upheno/imports/mpath_import.owl
http://purl.obolibrary.org/obo/upheno/imports/doid_import.owl
http://purl.obolibrary.org/obo/upheno/imports/extra.owl
http://purl.obolibrary.org/obo/upheno/imports/pr_import.owl


INFO:root:Read source with 579995 triples.


../data/phenotype/phenotype-hp-mp-2017-bioportal/hp-mp/target.rdf
http://purl.obolibrary.org/obo/upheno/imports/doid_import.owl
http://purl.obolibrary.org/obo/upheno/imports/extra.owl
http://purl.obolibrary.org/obo/upheno/imports/uberon_import.owl
http://purl.obolibrary.org/obo/upheno/imports/mpath_import.owl
http://purl.obolibrary.org/obo/upheno/imports/go_import.owl
http://purl.obolibrary.org/obo/upheno/imports/ro_import.owl
http://purl.obolibrary.org/obo/upheno/imports/pato_import.owl
http://purl.obolibrary.org/obo/upheno/imports/chebi_import.owl
http://purl.obolibrary.org/obo/upheno/imports/nbo_import.owl
http://purl.obolibrary.org/obo/upheno/imports/pr_import.owl


INFO:root:Read target with 561486 triples.
INFO:root:MAP CONCEPTS
INFO:root:The number of level_1 predicted mapping is 21054.
INFO:root:Retrieving Synsets by WordNet...
INFO:root:The number of level_2 predicted mapping is 14
INFO:root:Skip the level_3 mapping.
INFO:root:Skip the level_4 mapping.
INFO:root:Skip the level_5 mapping.
INFO:root:Matching Label Embeddings by Optimal Transport...
INFO:root:Computing the Ground Embedding Costs between the Source and Target Points...
INFO:root:The shape of the cost matrix is (12182, 11202)
INFO:root:Computing Optimal Transport Plan...
INFO:root:Computing Wasserstein distance by the Sinkhorn algorithm...
INFO:root:Making Mappings from a Pairwise OT Plan Matrix by Mutual NN...
INFO:root:The number of level_6 predicted mapping is 5344
INFO:root:TOTAL NUMBER OF MAPPINGS BETWEEN CONCPETS IS 26412
INFO:root:=================================================
INFO:root:MAP OBJECT PROPERTIES
INFO:root:The number of level_1 predicted mapping is 123.
INFO:

In [42]:
preds = maponto.load_alignments(output_url, 'predicted')
maponto.evaluate(preds, refs_url)

Total number of predicted is 26535
Total number of references is 696
Total correctly predicted alignments is 690
Total number of predicted is 26535
Precision is 0.026003391746749576
Recall is 0.9913793103448276
F1-Measure is 0.050677536631045496


# Geolink

In [14]:
source_url = "../data/geolink/geolink-v1/gbo-gmo/source.rdf"
target_url = "../data/geolink/geolink-v1/gbo-gmo/target.rdf"
output_url = '../data/results/geolink_gbo_gmo_alignment.rdf'
refs_url = "../data/geolink/geolink-v1/gbo-gmo/reference.rdf"

In [15]:
mapper.align(source_url, target_url, output_url)

INFO:root:Python ensemble mapper info: map ../data/geolink/geolink-v1/gbo-gmo/source.rdf to ../data/geolink/geolink-v1/gbo-gmo/target.rdf


ParserError: file:///Users/yan/Google%20Drive/django/otmaponto/match/ontology_mapping/data/geolink/geolink-v1/gbo-gmo/source.rdf:21:62: Repeat node-elements inside property elements: http://www.w3.org/2002/07/owl#IRI

In [ ]:
preds = maponto.load_alignments(output_url, 'predicted')
maponto.evaluate(preds, refs_url)

# Hydrography

In [16]:
source_url = "../data/hydrography/hydrography-v1/cree-swo/source.rdf"
target_url = "../data/hydrography/hydrography-v1/cree-swo/target.rdf"
output_url = '../data/results/hydrograph_cree_swo_alignment.rdf'
refs_url = "../data/hydrography/hydrography-v1/cree-swo/reference.rdf"

In [17]:
mapper.align(source_url, target_url, output_url)

INFO:root:Python ensemble mapper info: map ../data/hydrography/hydrography-v1/cree-swo/source.rdf to ../data/hydrography/hydrography-v1/cree-swo/target.rdf


ParserError: file:///Users/yan/Google%20Drive/django/otmaponto/match/ontology_mapping/data/hydrography/hydrography-v1/cree-swo/source.rdf:2638:35: Repeat node-elements inside property elements: http://www.w3.org/2002/07/owl#Class

In [ ]:
preds = maponto.load_alignments(output_url, 'predicted')
maponto.evaluate(preds, refs_url)

In [18]:
source_url = "../data/hydrography/hydrography-v1/hydro3-swo/source.rdf"
target_url = "../data/hydrography/hydrography-v1/hydro3-swo/target.rdf"
output_url = '../data/results/hydrograph_hydro3_swo_alignment.rdf'
refs_url = "../data/hydrography/hydrography-v1/hydro3-swo/reference.rdf"

In [19]:
mapper.align(source_url, target_url, output_url)

INFO:root:Python ensemble mapper info: map ../data/hydrography/hydrography-v1/hydro3-swo/source.rdf to ../data/hydrography/hydrography-v1/hydro3-swo/target.rdf


ParserError: file:///Users/yan/Google%20Drive/django/otmaponto/match/ontology_mapping/data/hydrography/hydrography-v1/hydro3-swo/source.rdf:15:101: Repeat node-elements inside property elements: http://www.w3.org/2002/07/owl#Literal

In [ ]:
preds = maponto.load_alignments(output_url, 'predicted')
maponto.evaluate(preds, refs_url)

# Popenslaved

In [35]:
source_url = "../data/popenslaved/popenslaved-v1/enslaved-wikidata/source.rdf"
target_url = "../data/popenslaved/popenslaved-v1/enslaved-wikidata/target.rdf"
output_url = '../data/results/popenslaved_enslaved_wikidata_alignment.rdf'
refs_url = "../data/popenslaved/popenslaved-v1/enslaved-wikidata/reference.rdf"

In [36]:
mapper.align(source_url, target_url, output_url)

INFO:root:Python ensemble mapper info: map ../data/popenslaved/popenslaved-v1/enslaved-wikidata/source.rdf to ../data/popenslaved/popenslaved-v1/enslaved-wikidata/target.rdf
INFO:root:Read source with 68133 triples.
INFO:root:Read target with 83515 triples.
INFO:root:MAP CONCEPTS
INFO:root:The number of level_1 predicted mapping is 8.
INFO:root:Retrieving Synsets by WordNet...
INFO:root:The number of level_2 predicted mapping is 1
INFO:root:Compute the related words for each source concept....
33it [00:00, 147.65it/s]
INFO:root:Total number of source concepts computed with relatd words is 33
INFO:root:Compute the related words for each target concept....
9it [00:00, 140.96it/s]
INFO:root:Total number of target concepts computed with related words is 9
100%|██████████| 33/33 [00:00<00:00, 40926.09it/s]
INFO:root:The number of level_3 predicted mapping is 6
INFO:root:Compute Pairwise Wasserstein Distances...
100%|██████████| 27/27 [00:00<00:00, 563.70it/s]
INFO:root:The number of level_4

49it [00:00, 146.99it/s]
INFO:root:Total number of source concepts computed with relatd words is 49
INFO:root:Compute the related words for each target concept....
0it [00:00, ?it/s]
INFO:root:Total number of target concepts computed with related words is 0
INFO:root:Skip the level_4 mapping.
INFO:root:Skip the level_5 mapping.
INFO:root:Skip the level_6 mappping.
INFO:root:TOTAL NUMBER OF MAPPINGS BETWEEN OBJECT PROPERTIES IS 0
INFO:root:========================================================
INFO:root:MAP DATATYPE PROPERTIES
INFO:root:Retrieving Synsets by WordNet...
INFO:root:Compute the related words for each source concept....
17it [00:00, 147.55it/s]
INFO:root:Total number of source concepts computed with relatd words is 17
INFO:root:Compute the related words for each target concept....
0it [00:00, ?it/s]
INFO:root:Total number of target concepts computed with related words is 0
INFO:root:Skip the level_4 mapping.
INFO:root:Skip the level_5 mapping.
INFO:root:Skip the level_6 ma

In [37]:
preds = maponto.load_alignments(output_url, 'predicted')
maponto.evaluate(preds, refs_url)

Total number of predicted is 18
Total number of references is 98


KeyError: 'rdf:resource'

# Popgeolink

In [23]:
source_url = "../data/popgeolink/popgeolink-v1/popgbo-popgmo/source.rdf"
target_url = "../data/popgeolink/popgeolink-v1/popgbo-popgmo/target.rdf"
output_url = '../data/results/popgeolink_popgbo_popgmo_alignment.rdf'
refs_url = "../data/popgeolink/popgeolink-v1/popgbo-popgmo/reference.rdf"

In [24]:
mapper.align(source_url, target_url, output_url)

INFO:root:Python ensemble mapper info: map ../data/popgeolink/popgeolink-v1/popgbo-popgmo/source.rdf to ../data/popgeolink/popgeolink-v1/popgbo-popgmo/target.rdf


ParserError: file:///Users/yan/Google%20Drive/django/otmaponto/match/ontology_mapping/data/popgeolink/popgeolink-v1/popgbo-popgmo/source.rdf:21:62: Repeat node-elements inside property elements: http://www.w3.org/2002/07/owl#IRI

In [ ]:
preds = maponto.load_alignments(output_url, 'predicted')
maponto.evaluate(preds, refs_url)